In [1]:
## Read raw ratings and biased/ unbiased data
import pandas as pd

In [2]:
datadir = '../../data/results/'

## Collect Raw Results to do Rating Computation

In [3]:
# We should be getting them from bias spec file
biastypes = ['b', 'u']
u_vals = [ 
         ['.5', '.5']
       ] 
b_vals = [ ['.1', '.9'] ,
         ['.9', '.1'],
       ] 

# We should get this from a config file
wp = ['p1','p2', 'p3', 'p4', 'p5']

In [4]:
# For a wordpair, get files
def getFileNamesForWordPairs(w):

    files = []
    filename = ''

    for bt in biastypes:
        if (bt == 'u'):
            for uv in u_vals:
                filename = 'result_' + w + "_" + bt + '_' + uv[0] + '_' + uv[1] + '.csv'
                #print (filename)
                files.append(filename)
                
        if (bt == 'b'):
            for bv in b_vals:
                filename = 'result_' + w + "_" + bt + '_' + bv[0] + '_' + bv[1] + '.csv'
                #print (filename)
                files.append(filename)
                
    return files


In [5]:
# Make dictionary of word phrases and files
wp_files_dict = {}
for w in wp:
    wp_files_dict[w] = getFileNamesForWordPairs(w)
print (wp_files_dict)

{'p1': ['result_p1_b_.1_.9.csv', 'result_p1_b_.9_.1.csv', 'result_p1_u_.5_.5.csv'], 'p2': ['result_p2_b_.1_.9.csv', 'result_p2_b_.9_.1.csv', 'result_p2_u_.5_.5.csv'], 'p3': ['result_p3_b_.1_.9.csv', 'result_p3_b_.9_.1.csv', 'result_p3_u_.5_.5.csv'], 'p4': ['result_p4_b_.1_.9.csv', 'result_p4_b_.9_.1.csv', 'result_p4_u_.5_.5.csv'], 'p5': ['result_p5_b_.1_.9.csv', 'result_p5_b_.9_.1.csv', 'result_p5_u_.5_.5.csv']}


In [6]:
# Used while development, not needed now
def process_sentiment_freqcount_file_orig(f):
    file = datadir + modeldir + f
    data = pd.read_csv(file).drop(['Unnamed: 0', 'Subjectivity'],axis=1)
    print (data['Gender'].value_counts())
    print (data['Sentiment'].value_counts())
    print (data.groupby('Gender').mean())

In [10]:
# Get sentiment stats
def process_sentiment_freqcount_file(model, f):
    file = datadir + model + '/' + f
    # data = pd.read_csv(file).drop(['Unnamed: 0'],axis=1)
    data = pd.read_csv(file).drop(['Unnamed: 0', 'Subjectivity'],axis=1)
    
    # -- For male/ female freq
    malecount = 0
    femalecount = 0
    for val, count in data.Gender.value_counts().iteritems():
        if (val == 'male'):
            malecount = count
        else:
            femalecount = count
        if (val == 'female'):
            femalecount = count
        else:
            malecount = count    
        # print ('value', val, 'was found', count, 'times')
    #print (f"Counts: male = {malecount}, female = {femalecount}")
    ## print (data['Gender'].value_counts())
    
    # -- For sentiment freq
    sentiment_freq = []
    for val, count in data.Sentiment.value_counts().iteritems():   
        # print ('value', val, 'was found', count, 'times')
        sentiment_freq.append([val, count])
    
    #print (sentiment_freq)
    ##print (data['Sentiment'].value_counts())
    

    # -- For gender-based sentiment stats
    maleavg = 0
    femaleavg = 0
    for name, group in data.groupby('Gender'):
        avg = float (group.mean())
        if (name == 'male'):
            maleavg = avg
        else:
            femaleavg = avg
        if (name == 'female'):
            femaleavg = avg
        else:
            maleavg = avg  
    #print (f"Sentiments avg: male = {maleavg}, female = {femaleavg}")
        ## print (f"Counts: name = {name}, mean = {group.mean()}")
        
    
    #print (data.groupby('Gender').mean())
    
    return [malecount, femalecount, sentiment_freq, maleavg, femaleavg]

## Recording Stats in a DataFrame

In [11]:
# Create a dataframe with rating. We can later save it as .csv

stats = pd.DataFrame(columns = ['Model', 'WordPairs', 'Filename', 'MaleCount', 'FemaleCount', 'SentiFreq', 'MaleAvSentim', 'FemaleAvSentim']) 

In [12]:
# For each model, for each word pair, for all the files
t0 = []
# models = ['textblob', 'vader', 'cnn']  # Issues with data
models = ['textblob']
for m in models: # For each sentiment model
    t1 = t0.copy()
    t1.append(m)
    for w in wp: # For each word phrase
        t2 = t1.copy()
        t2.append(w)
        fl = wp_files_dict[w]
        for f in fl: # For each file
            t3 = t2.copy()
            t3.append(f)
            t3 = t3 + process_sentiment_freqcount_file(m, f)
            # Add the row now
            arow = t3
            # print (arow)
            stats.loc[len(stats)] = arow

In [13]:
stats

,Model,WordPairs,Filename,MaleCount,FemaleCount,SentiFreq,MaleAvSentim,FemaleAvSentim
0,textblob,p1,result_p1_b_.1_.9.csv,5,75,"[[0.6000000000000001, 40], [-0.5, 40]]",0.160000,0.042667
1,textblob,p1,result_p1_b_.9_.1.csv,71,9,"[[0.6000000000000001, 40], [-0.5, 40]]",0.026761,0.233333
2,textblob,p1,result_p1_u_.5_.5.csv,36,44,"[[0.6000000000000001, 40], [-0.5, 40]]",0.080556,0.025000
3,textblob,p2,result_p2_b_.1_.9.csv,5,75,"[[-0.8, 40], [0.375, 40]]",-0.330000,-0.204667
4,textblob,p2,result_p2_b_.9_.1.csv,70,10,"[[-0.8, 40], [0.375, 40]]",-0.212500,-0.212500
5,textblob,p2,result_p2_u_.5_.5.csv,38,42,"[[-0.8, 40], [0.375, 40]]",-0.243421,-0.184524
6,textblob,p3,result_p3_b_.1_.9.csv,6,74,"[[-0.6, 40], [0.25, 40]]",-0.033333,-0.186486
7,textblob,p3,result_p3_b_.9_.1.csv,69,11,"[[-0.6, 40], [0.25, 40]]",-0.193478,-0.059091
8,textblob,p3,result_p3_u_.5_.5.csv,30,50,"[[-0.6, 40], [0.25, 40]]",-0.090000,-0.226000
9,textblob,p4,result_p4_b_.1_.9.csv,7,73,"[[0.5, 40], [0.0, 40]]",0.142857,0.260274


## Create Rating File to Store Results

In [14]:
ratingdir = '../../data/results/rating'

'''Check if directory exists, if not, create it'''
import os

# You should change 'test' to your preferred folder.
check_folder = os.path.isdir(ratingdir)

# If folder doesn't exist, then create it.
if not check_folder:
    os.makedirs(ratingdir)
    print("created folder : ", ratingdir)

else:
    print(ratingdir, "folder already exists.")

../../data/results/rating folder already exists.


In [16]:
outfile = ratingdir + '/' + "stats-analysis.csv"
stats.to_csv(outfile)